### An example implementation of the CAREforMe pipeline using the WESAD dataset

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../care_for_me"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import numpy as np
import pandas as pd
import wesad

from care_for_me import signals
from sklearn.metrics import accuracy_score, f1_score

subject_indices = list(range(2, 12)) + list(range(13, 18))
SUBJECTS = [str(i) for i in subject_indices]

# NOTE: Change ROOT_DIR according to your own file structure. This will be the only place you will need to do this.
ROOT_DIR = "/Users/emilyzhou/Desktop/Research/CAREForMe/"
DATA_DIR = os.path.join(ROOT_DIR, "data")
WESAD_PATH = os.path.join(DATA_DIR, "WESAD")
SOURCE_FOLDER = os.path.join(WESAD_PATH, "formatted")
ANNOTATIONS_PATH = os.path.join(WESAD_PATH, "annotations")
METRICS = os.path.join(DATA_DIR, "metrics", "WESAD")

In [9]:
# Format data to be compatible with CAREforMe pipeline
# Only needs to be run once locally 
# Status: COMPLETE, do not re-run

import glob
import os


# wesad.reformat_and_save_data(WESAD_PATH)

# Rename Medi_1 and Medi_2 files to Medi1 and Medi2
files_to_rename = glob.glob(os.path.join(SOURCE_FOLDER, "*/*Medi_*.csv"))
for file in files_to_rename:
    file_name = file.split("_")
    new_name = f"{file_name[0]}_{file_name[1]}{file_name[2]}_{file_name[3]}"
    os.rename(file, new_name)

In [2]:
labels = wesad.generate_labels(ANNOTATIONS_PATH, threshold="dynamic")
# print(labels)

def generate_labels(data):
    """
    Generate binary labels for WESAD based on the STAI questionnaire and the input data format.
    
    Parameters
    --------------------
    :param data: Features to generate labels for. Must include subject ID and phase columns.
    :type data: pd.DataFrame

    Returns
    --------------------
    Generated labels and the unmodified input data.
    """
    annotations = wesad.generate_labels(ANNOTATIONS_PATH, threshold="dynamic")
    labels = []
    for i in range(data.shape[0]):
        subject = int(data["subject"].iloc[i])
        phase = data["Phase"].iloc[i]
        label_row = annotations.loc[(annotations["subject"] == subject)]
        label = label_row[phase]
        labels.append(label)
    labels = np.array(labels).ravel()
    return labels, data

### Binary stress classification

In [4]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

label_gen = generate_labels
signal_acq = SignalAcquisition(signal_types=signal_types, source_folder=SOURCE_FOLDER)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=0)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...
- Elapsed time: 0.369 s
Running node Feature Extractor...


100%|██████████| 15/15 [00:15<00:00,  1.06s/it]

- Elapsed time: 15.944 s
Running node Label Generator...
- Elapsed time: 0.051 s
Running node Feature Selector...
- Elapsed time: 0.125 s
Running node SVC train-val-test...
Cross-validation scores: [0.83333333 0.75       0.66666667 1.         0.83333333]
- Elapsed time: 0.009 s

Accuracy: 0.9333333333333333


### Subject identification

In [6]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

labels = os.path.join(WESAD_PATH, "ANNOTATIONS_PATH")
label_gen = "subject"
signal_acq = SignalAcquisition(signal_types=signal_types, source_folder=SOURCE_FOLDER)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=0)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...
- Elapsed time: 0.376 s
Running node Feature Extractor...


100%|██████████| 15/15 [00:16<00:00,  1.08s/it]


- Elapsed time: 16.208 s
Running node Label Generator...
- Elapsed time: 0.0 s
Running node Feature Selector...
- Elapsed time: 0.209 s
Running node SVC train-val-test...
Cross-validation scores: [0.25       0.25       0.08333333 0.25       0.16666667]
- Elapsed time: 0.015 s

Accuracy: 0.06666666666666667


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


### Phase identification

In [12]:
# Build pipeline with default preprocessing and feature extraction methods

from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.label_generator.label_generator import LabelGenerator
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.classification.estimator import Estimator
from care_for_me.pipeline.pipeline import Pipeline

from sklearn.svm import SVC


signal_types = [
    signals.Signals.ECG,
    signals.Signals.EDA
]
feature_names = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

labels = os.path.join(WESAD_PATH, "ANNOTATIONS_PATH")
label_gen = "phase"
signal_acq = SignalAcquisition(signal_types=signal_types, source_folder=SOURCE_FOLDER)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=0)
feature_extractor = FeatureExtractor()
label_generator = LabelGenerator(label_generation_method=label_gen)

model = SVC()
feature_selector = FeatureSelector(model, feature_names, num_features=3)
estimator_train = Estimator(0, model, name="SVC training")
estimator_test = Estimator(1, model, name="SVC testing")
estimator_train_val_test = Estimator(2, model, name="SVC train-val-test")

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    # [signal_acq, signal_preprocessor, feature_extractor]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector]
    # [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train, estimator_test]
    [signal_acq, signal_preprocessor, feature_extractor, label_generator, feature_selector, estimator_train_val_test]
)

# We leave it up to the user to handle the final output of the pipeline. 
out = pipeline.run()

# Results
# fitted_model = out[0]
y_true = out[1]
y_pred = out[2]

acc = accuracy_score(y_true, y_pred)
# f1 = f1_score(y_true, y_pred)

print(f"\nAccuracy: {acc}")
# print(f"F1-score: {f1}")

Running node Signal Acquisition...
- Elapsed time: 0.0 s
Running node Signal Preprocessor...
- Elapsed time: 0.375 s
Running node Feature Extractor...


100%|██████████| 15/15 [00:15<00:00,  1.07s/it]

- Elapsed time: 15.993 s
Running node Label Generator...
- Elapsed time: 0.0 s
Running node Feature Selector...
- Elapsed time: 0.093 s
Running node SVC train-val-test...
Cross-validation scores: [0.16666667 0.16666667 0.16666667 0.25       0.16666667]
- Elapsed time: 0.011 s

Accuracy: 0.06666666666666667
